# Tasks:

### 1) Load data from Binance: https://www.binance.com/en/landing/data:

![Load klines](attachment/load_klines.png)

### 2) Calculate and plot indicators

1. Moving average (MA)
2. Smoothed moving average (SMA)
...

### 3) Using historical market data and copytrader trades try to find strategy. 
https://www.binance.com/en/copy-trading/lead-details/3838699803249970688

https://www.binance.com/en/copy-trading/lead-details/3871399360910422273

In [6]:
import pandas as pd

copytrader_1_trades = pd.read_csv('copytrader_data/trades_3838699803249970688.csv')
copytrader_2_trades = pd.read_csv('copytrader_data/trades_3871399360910422273.csv')

copytrader_1_trades.head(10)

,Unnamed: 0,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,0,1714060960000,DOGEUSDT,SELL,0.15192,-0.060768,USDT,303.84000,USDT,1.954933,USDT,DOGE,2000.0,LONG,True
1,1,1714060918000,DOGEUSDT,SELL,0.15182,-0.060728,USDT,303.64000,USDT,1.754933,USDT,DOGE,2000.0,LONG,True
2,2,1714060916000,DOGEUSDT,SELL,0.15172,-0.060688,USDT,303.44000,USDT,1.554933,USDT,DOGE,2000.0,LONG,True
3,3,1714060301000,DOGEUSDT,SELL,0.15162,-0.060648,USDT,303.24000,USDT,1.354933,USDT,DOGE,2000.0,LONG,True
4,4,1714060274000,DOGEUSDT,SELL,0.15152,-0.060608,USDT,303.04000,USDT,1.154933,USDT,DOGE,2000.0,LONG,True
5,5,1714060269000,DOGEUSDT,SELL,0.15142,-0.028316,USDT,141.57770,USDT,0.446431,USDT,DOGE,935.0,LONG,True
6,6,1714060269000,DOGEUSDT,SELL,0.15142,-0.032252,USDT,161.26230,USDT,0.508502,USDT,DOGE,1065.0,LONG,True
7,7,1714060250000,DOGEUSDT,SELL,0.15132,-0.060528,USDT,302.64000,USDT,0.754933,USDT,DOGE,2000.0,LONG,True
8,8,1714060215000,DOGEUSDT,SELL,0.15098,-0.051862,USDT,103.72326,USDT,0.025740,USDT,DOGE,687.0,LONG,False
9,9,1714060215000,DOGEUSDT,SELL,0.15098,-0.049974,USDT,99.94876,USDT,0.024803,USDT,DOGE,662.0,LONG,False
